In [38]:
import mysql.connector
import pandas as pd
import requests
import json
from datetime import date
import datetime
import time

### Download Data from the Polygon

- Meta (META)
- Apple (APPL)
- Amazon (AMZN)
- Netflix (NFLX)
- Alphabet (GOOG)
- Microsoft (MSFT)

In [39]:
def get_company_data(ticker, start_day, end_day):
    request_string='https://api.polygon.io/v2/aggs/ticker/'+ticker+'/range/1/day/'+start_day+'/'+end_day+'?adjusted=true&sort=asc&apiKey=46ipIrH5BrzkTqZj7OUvik7XAFOpu8GJ'

    api_response = requests.get(request_string)
    data = api_response.text
    return json.loads(data)

In [40]:
def insert_data(ticker, row, local_cursor, cnx):
    symbol = ticker
    local_date = str(row['t'])[0:-3]
    ticker_date = datetime.datetime.fromtimestamp(int(local_date)).date()
    open_price = row['o']
    high_price = row['h']
    low_price = row['l']
    close_price = row['c']
    volume = row['v']

    print(symbol, ticker_date, open_price, high_price, low_price, close_price, volume)
                     
    sql = "INSERT INTO stock (symbol, ticker_date, open_price, high_price, low_price, close_price, volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    val = (symbol, ticker_date, open_price, high_price, low_price, close_price, volume)
    local_cursor.execute(sql, val)
    cnx.commit()

In [41]:
#You need to set up your connection with your credentials
connection = mysql.connector.connect(user='stock_user', password='letmein',
                              host='127.0.0.1',
                              database='stock_data')
global_cursor = connection.cursor()

In [77]:
ticker='META'
start_day='2022-06-09'
end_day='2023-02-10'

In [78]:
data = get_company_data(ticker, start_day, end_day)['results']

In [79]:
for i in range(0, len(data)):
    insert_data(ticker, data[i], global_cursor, connection)

META 2022-06-09 194.28 199.45 183.68 184 23501579.0
META 2022-06-10 183.04 183.1 175.02 175.57 27478354.0
META 2022-06-13 170.59 172.575 164.03 164.26 31749304.0
META 2022-06-14 166.03 166.75 161.3601 163.73 27192049.0
META 2022-06-15 167.2 172.16 163.975 169.35 29808251.0
META 2022-06-16 163.72 165.08 159.61 160.87 26944143.0
META 2022-06-17 161.68 165.9 159.9 163.74 31317515.0
META 2022-06-21 165.93 167.76 155.97 157.05 50661882.0
META 2022-06-22 158.38 160.79 155.44 155.85 47596820.0
META 2022-06-23 156.27 159.75 154.25 158.75 40499198.0
META 2022-06-24 161.725 170.25 161.3 170.16 68736009.0
META 2022-06-27 171.32 171.75 168.01 169.49 29167509.0
META 2022-06-28 169.9 171.3 160.61 160.68 27788725.0
META 2022-06-29 163.57 166.33 160.32 163.94 28595236.0
META 2022-06-30 162.15 165.23 158.49 161.25 35150605.0
META 2022-07-01 160.31 160.62 155 160.03 31364183.0
META 2022-07-05 158.14 168.25 157.02 168.19 28664799.0
META 2022-07-06 168.2 171.59 165.46 169.77 23085921.0
META 2022-07-07 169

In [80]:
connection.close()